#### Fill in the blank to complete the following code and run:

In [1]:
import io
import base64
import numpy as np
from PIL import Image
import tensorflow as tf
import json
from flask import Flask, render_template, request
import requests

In [2]:
app = Flask(__name__, template_folder="./Website")

img_height = 224
img_width = 224

def loadmodel():    
    with open("./model/InceptionV3Adam.json","r") as json_file:
        loaded_model_json= json_file.read()
        
    #Load structure of model from json
    loaded_model= tf.keras.models.model_from_json(loaded_model_json)    
    #Load weight from model.h5 file
    loaded_model.load_weights("./model/InceptionV3Adam_weights.h5")
    
    return loaded_model

# Function to prepare the image data
def prepare_image(img, img_url=None):
    # If an image URL is provided, download the image
    if img_url:
        response = requests.get(img_url)
        img = Image.open(io.BytesIO(response.content))
    else:
        img = Image.open(io.BytesIO(img))
        
    # Convert the image to RGB format
    img = img.convert("RGB")
    # Resize the image to the specified height and width
    img = img.resize((img_height, img_width)) 
    # Convert the image to a numpy array and normalize the pixel values
    img = np.array(img)
    img = img/255.0
    # Reshape the image data for the model
    img = img.reshape(img_height, img_width, 3)   
    img = np.expand_dims(img, 0)
    return img

# Function to make a prediction on the image
def predict_result(img):
    # Class names for the 4 classes
    class_names = ["drum", "guitar", "piano", "violin"]

    # Call the loadmodel function to get the model
    model= loadmodel()
    # Make a prediction on the image data
    rpredict= model.predict(img)

    # Return the class name with the highest probability
    return class_names[np.argmax(rpredict[0])]

@app.route("/")
def index():
    return render_template("Home.html")

@app.route("/submiturl", methods=["GET", "POST"])
def submiturl():
    if request.method == "POST":
        # Get the image URL from the form
        url = request.form.get("img_url")
        
        # Check if the URL is not empty
        if url:
            # Prepare the image data
            img = prepare_image(None, url)
            
            # Classify the image data
            result = predict_result(img)
            
            # Render the result page with the image URL and result
            return render_template("Url.html", result=result, img_url=url), 200
        else:
            # Return an error message if the URL is empty
            return "Please try again. The image URL is empty."
    else:
        return render_template("Url.html")

    
@app.route("/submitimage", methods=["GET", "POST"])
def submitimage():
    if request.method == "POST":
        if "imagefile" not in request.files:
            return "Please try again. The image does not exist."
        
        # Get the image file from request.files
        file = request.files["imagefile"]

        if not file:
            return "Please try again. No image file received."
        
          # Read image into bytes
        img_bytes = file.read()
        
        # Convert image bytes into base64 string
        img_string = base64.b64encode(img_bytes).decode()
        
        # Prepare image data
        img = prepare_image(img_bytes)
        
        # Classify the image data
        result = predict_result(img)

        return render_template("Image.html", img_data=img_string, prediction=result), 200
    else:
        # Render the template for GET requests
        return render_template("Image.html")

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    app.debug= True    
    run_simple("localhost", 5009, app)

 * Running on http://localhost:5009
Press CTRL+C to quit
127.0.0.1 - - [11/Feb/2023 11:52:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2023 11:52:14] "GET /submitimage HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2023 11:52:24] "POST /submitimage HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2023 11:52:36] "POST /submitimage HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2023 11:52:39] "GET /submiturl HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2023 11:52:47] "POST /submiturl HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2023 11:52:57] "POST /submiturl HTTP/1.1" 200 -
